# インターネット上から情報を収集するsample

## 内容

- Pythonを使用してインターネット上から情報を収集する

## 参考URL

- Pythonのスクレイピングで文字だけを抜き出す汎用的な方法
    - https://qiita.com/poorko/items/9140c75415d748633a10

- 【Webスクレイピング入門】Google検索の上位サイトを件数指定して表示する方法
    - https://rurukblog.com/post/WebScraping-Google-Top/

- 【コピペOK】PythonでGoogle検索する方法【BeautifulSoup】
    - http://kakedashi-xx.com:25214/index.php/2020/12/19/post-1565/

環境は下記URLにアクセスすると確認できます。
(User-Agentは下記画像のブラウザのユーザーエージェント HTTP_USER_AGENTで取得できます)
https://testpage.jp/tool/ip_user_agent.php

# Load modules

In [ ]:
import sys, os
import gc
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup

# Constants

## paths

In [ ]:
input_dpath = '../input/'
output_dpath = '../output/'

# Functions

## get_soup

In [ ]:
def get_soup(url):
    """
    URLからBeautifulSoupのインスタンスを作成する。requests.get使用。

    Args:
        url: str
            soupインスタンスを作成したいURL
    
    Returns:
        soup: bs4.BeautifulSoup instance
            soupインスタンス。
    """
    # htmlを取得
    html = requests.get(url)

    # soupを作成
    soup = BeautifulSoup(html.content, 'html.parser')

    return soup

## gettext_fromurl

In [ ]:
def gettext_fromurl(url):
    """
    urlで指定したページのtitleとbodyのtextを抜き出す。

    Args:
        url: str
            titleとtextを抜き出したいページのURL。
    
    Returns:
        title, text: tuple
            title: str
                ページのhtmlのheaderのtitle。
                取得に失敗した時は'get title error'
            
            text: str
                ページのhtmlのbodyのtext。
                取得に失敗した時は'get text error'
    """
    url = str(url)
    try:
        sp = get_soup(url)
    except:
        print('get text error:', url)
        print()
        return 'get title error', 'get text error'
    
    # title
    title = sp.title.string
    title = str(title)
    
    # bodyのtext
    for script in sp(['script', 'style']):
        script.decompose()
    text = sp.get_text()
    lines = [line.strip() for line in text.splitlines()]
    text="\n".join(line for line in lines if line)
    return title, text

## download_img

In [ ]:
def download_img(url, file_name='./img.jpg', force=False):
    """
    画像をダウンロードする。requests.get使用。

    Args:
        url: str
            ダウンロードする画像ファイルのURL。
        
        file_name: str, optional(default='./img.jpg')
            ダウンロードした後の画像ファイルの名前。パスを含む。
        
        force: bool, optional(default=False)
            force to overwrite. 強制的に上書きするかどうか。
            True: 強制的に上書きする。
            False: すでにファイルが存在していれば保存しない。
    """
    file_name = str(file_name)

    if not force:
        # すでにファイルが存在していれば保存しない
        if os.path.exists(file_name):
            print(file_name + ' is already exists.')
            return

    # 保存する
    r = requests.get(url, stream=True)
    if r.status_code == 200:
        with open(file_name, 'wb') as f:
            f.write(r.content)

## search_google

In [ ]:
def search_google(google_search_url, n_pages=11):
    """
    google検索を行う．
    
    Args:
        google_search_url: str
            Googleから検索結果ページ
        
        n_pages: str
            上位から何件までのサイトを抽出するか
    
    Returns:
        search_result_df: pandas.DataFrame
    """
    request = requests.get(google_search_url)

    # Googleのページ解析を行う
    soup = BeautifulSoup(request.text, "html.parser")
    search_pages_list = soup.select('div.kCrYT > a')

    search_result_list = []

    # ページ解析と結果の出力
    for rank, page in zip(range(1, n_pages), search_pages_list):
        try:
            try:
                page_title1 = page.select('h3.zBAuLc')[0].text
            except IndexError:
                page_title1 = page.select('img')[0]['alt']

            page_url = page['href'].replace('/url?q=', '')
            page_url = page_url.split('&sa=')[0]
            # print(page_url)
            page_title2, page_text = gettext_fromurl(page_url)

        except Exception as e:
            continue

        search_result_list += [[
            rank,
            page_url,
            page_title1,
            page_title2,
            page_text
        ]]

    search_result_df = pd.DataFrame(
        search_result_list,
        columns=[
            'page_rank',
            'page_url',
            'page_title1',
            'page_title2',
            'page_text'
        ]
    )

    return search_result_df

# クローリングsample

## 検索ワード等の設定

In [ ]:
# Google検索するキーワードを設定
search_word = 'python'

# 上位から何件までのサイトを抽出するか指定する
n_pages = 10 + 1

# Googleから検索結果ページを取得する
google_search_url = f'https://www.google.co.jp/search?hl=ja&num={n_pages}&q={search_word}'

## 検索を行う

In [ ]:
search_result_df = search_google(
    google_search_url,
    n_pages
)

In [ ]:
search_result_df.head()

# 「松江 住みやすさ」で検索してみる

## 検索ワード等の設定

In [ ]:
# Google検索するキーワードを設定
search_word = '松江+住みやすさ'

# 上位から何件までのサイトを抽出するか指定する
n_pages = 10 + 1

# Googleから検索結果ページを取得する
google_search_url = f'https://www.google.co.jp/search?hl=ja&num={n_pages}&q={search_word}'

## 検索を行う

In [ ]:
matsue_confort_df = search_google(
    google_search_url,
    n_pages
)

In [ ]:
matsue_confort_df.head()

## 結果を出力する

### ファイルパスの設定

In [ ]:
confort_fname = '住みやすさ検索結果'

confort_excel_fpath = f'{output_dpath}{confort_fname}.xlsx'
confort_csv_fpath = f'{output_dpath}{confort_fname}.csv'
confort_notext_csv_fpath = f'{output_dpath}{confort_fname}_notext.csv'

print('confort_excel_fpath:', confort_excel_fpath)
print('confort_csv_fpath:', confort_csv_fpath)

### excelとcsvの出力

In [ ]:
matsue_confort_df.to_excel(confort_excel_fpath, index=True)

In [ ]:
matsue_confort_df.to_csv(confort_csv_fpath, index=False)

In [ ]:
matsue_confort_df.drop(
    labels=['page_text'],
    axis=1
).to_csv(confort_notext_csv_fpath, index=False)